# Step 3D: BERT Model 4 - BERT sentiment analysis using finBERT

## 1. Required Imports

### 1.1 Import required libraries
*Ignore the warning about audio backend since it is not required for our purpose*

In [1]:
import pandas as pd
import re
from transformers import pipeline

C:\Users\Karthik\AppData\Roaming\Python\Python38\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


### 1.2 Import reddit and twitter data for which we have to populate the sentiments
- *we will use them on the model after the model is trained with the training dataset from kaggle*
- *we save data in two dictionaries; one for reddit and one for twitter to make it more organized*### 1.3 Import reddit and twitter data for which we have to populate the sentiments
- *we will use them on the model after the model is trained with the training dataset from kaggle*

In [2]:
tick_list =  ['AAPL', 'AMC', 'DKNG', 'TSLA', 'AMD', 'BABA']

In [3]:
reddit_df_dict = {tick: pd.read_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Reddit\\consolidated_pickle_files\\reddit_{tick}_df_for_BERT.pkl") for tick in tick_list}
# reddit_df_dict  

In [4]:
reddit_df_dict['TSLA']

,body,created_utc,id,top,year,month,day,ticker,YearMonDay
0,Non-troll post. I started 2 weeks ago and have...,1609500199,ghp72zs,top,21,Jan,01,TSLA,21Jan01
1,we eat cornbread on new years day to ensure a ...,1609524808,ghqo6qb,top,21,Jan,01,TSLA,21Jan01
2,TSLA 850 EOD,1612203941,glmp0gv,top,21,Feb,01,TSLA,21Feb01
3,joe weisenthal is the ultimate chad( who else ...,1612207510,glmyg66,top,21,Feb,01,TSLA,21Feb01
4,TSLA 🚀🚀🚀🚀🚀,1612211972,gln9v98,top,21,Feb,01,TSLA,21Feb01
...,...,...,...,...,...,...,...,...,...
15125,TSLA Drill Team 6 reporting for duty,1609359381,ghj65jr,top,20,Dec,30,TSLA,20Dec30
15126,TSLA’s still having them TSLA days I see. Join...,1609359393,ghj66f1,top,20,Dec,30,TSLA,20Dec30
15127,TSLA 🎰 EOY coming,1609360555,ghj8jam,top,20,Dec,30,TSLA,20Dec30
15128,Thank fuck I didn't dump my TSLA calls yesterday,1609361591,ghjamup,top,20,Dec,30,TSLA,20Dec30


In [5]:
twitter_df_dict = {tick: pd.read_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Twitter\\consolidated_pickle_files\\twitter_{tick}_df_for_BERT.pkl") for tick in tick_list}

In [6]:
twitter_df_dict['TSLA']

,date,content,likeCount,lang,cleaned_content,year,month,day,ticker,YearMonDay
0,2021-09-18 19:56:59+00:00,@The_RockTrading Bullish on $TSLA this week &a...,1,en,Bullish on $TSLA this week & $Aapl,21,Sep,18,TSLA,21Sep18
1,2021-09-18 04:16:52+00:00,$TSLA now is the same as $aapl in the 80s ! @e...,1,en,$TSLA now is the same as $aapl in the 80s !,21,Sep,18,TSLA,21Sep18
2,2021-09-16 19:01:28+00:00,"Added more $TSLA and $aapl to long, because I ...",0,en,"Added more $TSLA and $aapl to long, because I ...",21,Sep,16,TSLA,21Sep16
3,2021-09-16 15:23:53+00:00,@NeilRog49855230 @Gays4Tesla @TheMaverickWS Th...,3,en,There is plenty of information available on-li...,21,Sep,16,TSLA,21Sep16
4,2021-09-16 15:23:11+00:00,There is plenty of information available on-li...,0,en,There is plenty of information available on-li...,21,Sep,16,TSLA,21Sep16
...,...,...,...,...,...,...,...,...,...,...
179489,2020-06-01 18:42:22+00:00,Normally volatile Tesla $TSLA is the IBD Stock...,1,en,Normally volatile Tesla $TSLA is the IBD Stock...,20,Jun,01,TSLA,20Jun01
179490,2020-06-01 18:42:10+00:00,Normally volatile Tesla $TSLA is the IBD Stock...,1,en,Normally volatile Tesla $TSLA is the IBD Stock...,20,Jun,01,TSLA,20Jun01
179491,2020-06-01 18:41:35+00:00,$TSLA up $51.00 from next suggested buy entry ...,1,en,$TSLA up $51.00 from next suggested buy entry ...,20,Jun,01,TSLA,20Jun01
179492,2020-06-01 18:41:32+00:00,@Desert_Trader81 $TSLA on the move crossing HO...,0,en,$TSLA on the move crossing HOD $885 !!!! 900 w...,20,Jun,01,TSLA,20Jun01


## 2 Prepare the datasets to be used in the BERT model for prediction

### 2.1 Clean the text data

In [7]:
def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text) #remove hashtags
    text = re.sub(r'http\S+', '', text)    #remove urls
    text = re.sub(r'&amp;amp', '&', text)  #remove double amps
    text = re.sub(r'\&amp;', '&', text)    #remove single amps
    text = re.sub(r'\s+', ' ', text)       #reduce multiple spaces into a single space
    text = re.sub(r'\s+-\s+', ' ', text)
    text = text.strip()
    return text

In [8]:
for tick, tick_df in reddit_df_dict.items():
#     print(tick, tick_df)
    tick_df['cleaned_body'] = tick_df['body'].apply(lambda x: text_preprocessing(x))
    tick_df['LEN'] = tick_df.cleaned_body.str.len()    

In [9]:
for tick, tick_df in twitter_df_dict.items():
    tick_df['cleaned_body'] = tick_df['cleaned_content'].apply(lambda x: text_preprocessing(x))
    tick_df['LEN'] = tick_df.cleaned_body.str.len()    

### 2.2 Handle records where text is greater than a length of 512
- *BERT can only handle a max length of 512*
- *For each comment > length of 512, break the comment into multiple sets of 512*
- *After breaking, create a new record for each of the broken parts and append to the original dataframe. Since we take the average sentiment scores, appending new rows with the same dates will not affect the data*

In [10]:
n=512
def ffn(xseries):
    list_of_lists = []
        
    if xseries['LEN']>512:
        parts = [xseries['cleaned_body'][i:i+n] for i in range(0, xseries['LEN'], n)]
    
        counter=0
        for i in parts:
            list_series = []
            list_series.append(xseries['body'])
            list_series.append(xseries['created_utc'])
            list_series.append(xseries['id'])
            list_series.append(xseries['top'])
            list_series.append(xseries['year'])
            list_series.append(xseries['month'])
            list_series.append(xseries['day'])
            list_series.append(xseries['ticker'])
            list_series.append(xseries['YearMonDay'])
            list_series.append(i)
            counter+=1
            list_series.append(counter)
            
            list_of_lists.append(list_series)
    
        return list_of_lists
    else:
        return 0

In [11]:
for tick, tick_df in reddit_df_dict.items():
    tick_df['mltp'] = tick_df.apply(ffn, axis=1)

In [12]:
for tick, tick_df in twitter_df_dict.items():
    tick_df['mltp'] = tick_df.apply(ffn, axis=1)

In [13]:
for tick, tick_df in reddit_df_dict.items():
    addition_list = tick_df.query('mltp != 0')['mltp']
    
    if len(addition_list)==0:
        continue
    
    ind_additions = []

    for list_of_sentences in addition_list:
        for sentence_part in list_of_sentences:
            ind_additions.append(sentence_part)
    
    
    df_additions = pd.DataFrame(ind_additions, columns=['body','created_utc','id','top','year','month','day','ticker','YearMonDay','cleaned_body','LEN'])
    
    tick_df.drop('mltp', axis=1, inplace=True)
    tick_df.drop(tick_df[tick_df.LEN > 512].index, inplace = True)
    tick_df = tick_df.append(df_additions, ignore_index=True)

In [14]:
for tick, tick_df in twitter_df_dict.items():
    addition_list = tick_df.query('mltp != 0')['mltp']
    
    if len(addition_list)==0:
        continue
    
    ind_additions = []

    for list_of_sentences in addition_list:
        for sentence_part in list_of_sentences:
            ind_additions.append(sentence_part)
    
    
    df_additions = pd.DataFrame(ind_additions, columns=['body','created_utc','id','top','year','month','day','ticker','YearMonDay','cleaned_body','LEN'])
    
    tick_df.drop('mltp', axis=1, inplace=True)
    tick_df.drop(tick_df[tick_df.LEN > 512].index, inplace = True)
    tick_df = tick_df.append(df_additions, ignore_index=True)

- *Make sure there is no data where comment length is greater than 512*

In [15]:
for tick, tick_df in reddit_df_dict.items():
    print(tick_df.query('LEN > 512'))

Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []


In [16]:
for tick, tick_df in twitter_df_dict.items():
    print(tick_df.query('LEN > 512'))

Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []


## 2. Use finBERT model and get outputs for each comment

### 2.1 Create a huggingface pipeline with finBERT model
- *set return_all_scores=True so that we receive scores for all 3 sentiments (positive, negative and neutral)*
- *Ref: https://huggingface.co/transformers/main_classes/pipelines.html*

In [17]:
classifier = pipeline('sentiment-analysis', model='ProsusAI/finbert', return_all_scores =True)

### 2.2 Get the sentiments for each day

#### 2.2.1 Get the BERT output for reddit dataset

In [18]:
for tick, tick_df in reddit_df_dict.items():
    BERT_yield = tick_df['cleaned_body'].apply(lambda x: classifier(x))
    #---The output from finBERT is in the form of a list of list of dicts i.e. "[[{key: value}]]"
    #---We convert it to a dataframe to make it easier for further steps
    temp_list = []
    for i in BERT_yield:
        temp_dict={}
        for j in range(0,3):
            k = i[0][j]["label"]
            v = i[0][j]["score"]
            temp_dict[k]=v
        temp_list.append(temp_dict)

    BERT_output_df = pd.DataFrame(temp_list)
    complete_output_df = pd.concat([tick_df, BERT_output_df], axis=1)
    sentiment_df = complete_output_df.groupby(['ticker', 'YearMonDay'])[['positive', 'neutral', 'negative']].mean().reset_index()
    reddit_df_dict[tick] = sentiment_df
    sentiment_df.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\reddit_{tick}_finBERT.pkl")
    

#### 2.2.2 Get the BERT output for twitter dataset

In [19]:
for tick, tick_df in twitter_df_dict.items():
    BERT_yield = tick_df['cleaned_body'].apply(lambda x: classifier(x))
    #---The output from finBERT is in the form of a list of list of dicts i.e. "[[{key: value}]]"
    #---We convert it to a dataframe to make it easier for further steps
    temp_list = []
    for i in BERT_yield:
        temp_dict={}
        for j in range(0,3):
            k = i[0][j]["label"]
            v = i[0][j]["score"]
            temp_dict[k]=v
        temp_list.append(temp_dict)

    BERT_output_df = pd.DataFrame(temp_list)
    complete_output_df = pd.concat([tick_df, BERT_output_df], axis=1)
    sentiment_df = complete_output_df.groupby(['ticker', 'YearMonDay'])[['positive', 'neutral', 'negative']].mean().reset_index()
    twitter_df_dict[tick] = sentiment_df
    sentiment_df.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\twitter_{tick}_finBERT.pkl")
    

#### 2.2.3 Calculate combined predictions by combining both reddit and twitter sentiments

In [20]:
for ticker in tick_list:
    print(ticker)
    reddit_sentis = reddit_df_dict[ticker][['ticker', 'YearMonDay', 'neutral', 'positive', 'negative']]
    twitter_sentis = twitter_df_dict[ticker][['ticker', 'YearMonDay', 'neutral', 'positive', 'negative']]
    
#     print(reddit_sentis,twitter_sentis,pd.concat([reddit_sentis,twitter_sentis], axis=0,ignore_index=True))
    combined_sentis = pd.concat([reddit_sentis,twitter_sentis], axis=0,ignore_index=True)
    combined_sentis_grouped = combined_sentis.groupby(['ticker', 'YearMonDay'])[['neutral', 'positive', 'negative']].mean().reset_index()
    combined_sentis_grouped.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\combined_{ticker}_finBERT.pkl")
    print(combined_sentis_grouped)

AAPL
    ticker YearMonDay   neutral  positive  negative
0     AAPL    20Aug01  0.666970  0.201389  0.131641
1     AAPL    20Aug02  0.701786  0.187726  0.110488
2     AAPL    20Aug03  0.697561  0.167927  0.134512
3     AAPL    20Aug04  0.706635  0.177406  0.115958
4     AAPL    20Aug05  0.635346  0.134119  0.230535
..     ...        ...       ...       ...       ...
499   AAPL    21Sep26  0.730306  0.135970  0.133725
500   AAPL    21Sep27  0.709497  0.151975  0.138528
501   AAPL    21Sep28  0.633344  0.160007  0.206648
502   AAPL    21Sep29  0.687631  0.173307  0.139062
503   AAPL    21Sep30  0.659920  0.169118  0.170962

[504 rows x 5 columns]
AMC
    ticker YearMonDay   neutral  positive  negative
0      AMC    20Aug01  0.743704  0.131487  0.124809
1      AMC    20Aug02  0.680545  0.044179  0.275276
2      AMC    20Aug03  0.539083  0.185692  0.275225
3      AMC    20Aug04  0.594843  0.235051  0.170106
4      AMC    20Aug05  0.635311  0.262705  0.101984
..     ...        ...       ...

#Ref
https://huggingface.co/transformers/main_classes/pipelines.html